# Request VC from Authority

This notebook is used in combination with notebook `01_issue_VC_city.ipynb` (see Authority agent). Break points indicate when to switch notebooks.

#### Imports

In [3]:
%%javascript
document.title='City Agent'

<IPython.core.display.Javascript object>

In [4]:
%autoawait 
from aries_cloudcontroller import AriesAgentController
#import asyncio
import os
from pprintpp import pprint
from termcolor import colored

IPython autoawait is `on`, and set to use `asyncio`


#### Initialize agent controller of City

In [5]:
api_key = os.getenv("ACAPY_ADMIN_API_KEY")
admin_url = os.getenv("ADMIN_URL")

print(f"Initialising a controller with admin api at {admin_url} and an api key of {api_key}")
agent_controller = AriesAgentController(admin_url,api_key)

Initialising a controller with admin api at http://city-agent:3021 and an api key of adminApiKey


## 1 – Init Webhook Server and register Event Listeners for it
#### 1.1 – Start webhook server
Start a webhook server to be able to communicate with other agents

In [6]:
# Setup
webhook_port = int(os.getenv("WEBHOOK_PORT"))
webhook_host = "0.0.0.0"

# Listen on webhook server
await agent_controller.init_webhook_server(webhook_host, webhook_port)
print(f"Listening for webhooks from agent at http://{webhook_host}:{webhook_port}")

Listening for webhooks from agent at http://0.0.0.0:3010


#### 1.2 – Define listeners
Define listeners that are triggered when something happens on the webhook server

In [7]:
# Receive connection messages
def connections_handler(payload):
    state = payload['state']
    connection_id = payload["connection_id"]
    their_role = payload["their_role"]
    routing_state = payload["routing_state"]
    rfc_state = payload["rfc23_state"]
    
    print("----------------------------------------------------------")
    print("Connection Webhook Event Received")
    print("Connection ID : ", connection_id)
    print("State : ", state)
    print("Routing State : {routing} ({rfc})".format(routing=routing_state, rfc=rfc_state))
    if 'their_label' in payload: 
        print(f"Connection with : ", payload['their_label'])
    print("Their Role : ", their_role)
    print("----------------------------------------------------------")

    if state == "invitation":
        # Your business logic
        print("invitation")
    elif state == "request":
        # Your business logic
        print("request")

    elif state == "response":
        # Your business logic
        print("response")
    elif state == "active":
        # Your business logic
        print(colored("Connection ID: {0} is now active.".format(connection_id), "green", attrs=["bold"]))

        
## YOUR LISTENERS HERE
def holder_handler(payload):
    connection_id = payload['connection_id']
    exchange_id = payload['credential_exchange_id']
    state = payload['state']
    role = payload['role']
    print("\n---------------------------------------------------\n")
    print("Handle Issue Credential Webhook")
    print(f"Connection ID : {connection_id}")
    print(f"Credential exchange ID : {exchange_id}")
    print("Agent Protocol Role : ", role)
    print("Protocol State : ", state )
    print("\n---------------------------------------------------\n")
    print("Handle Credential Webhook Payload")
    
    if state == "offer_received":
        print("Credential Offer Recieved")
        proposal = payload["credential_proposal_dict"]
        print("The proposal dictionary is likely how you would understand and display a credential offer in your application")
        print("\n", proposal)
        print("\n This includes the set of attributes you are being offered")
        attributes = proposal['credential_proposal']['attributes']
        print(attributes)
        ## YOUR LOGIC HERE
    elif state == "request_sent":
        print("\nA credential request object contains the commitment to the agents master secret using the nonce from the offer")
        ## YOUR LOGIC HERE
    elif state == "credential_received":
        print("Received Credential")
        ## YOUR LOGIC HERE
    elif state == "credential_acked":
        ## YOUR LOGIC HERE
        credential = payload["credential"]
        print("Credential Stored\n")
        print(credential)
        
        print("\nThe referent acts as the identifier for retrieving the raw credential from the wallet")
        # Note: You would probably save this in your application database
        credential_referent = credential["referent"]
        print("Referent", credential_referent)
        
        
def messages_handler(payload):
    connection_id = payload["connection_id"]
    agent_controller.messaging.send_message(connection_id, "This is a response from Bob")
    print("Handle message", connection_id)
    pprint(payload)

In [8]:
# Init listener list 
listeners = []

# Add listeners defined in previous cell
connection_listener = {"handler": connections_handler, "topic": "connections"}
listeners.append(connection_listener)

holder_listener = {"topic": "issue_credential", "handler": holder_handler}
listeners.append(holder_listener)

message_listener = {"handler": messages_handler, "topic": "basicmessages"}
listeners.append(message_listener)

#### 1.3 – Register listeners with `agent_controller`

In [9]:
agent_controller.register_listeners(listeners)

## 2 – Establish a connection with Authority agent
A connection with the credential issuer (i.e., the authority agent) must be establieshed before a VC can be received. In this scenario, the agent requests a connection with the Authority to be certified as an official city agency. Thus, the city agent sends an invitation to the Authority.

### 2.1 Create invitation to Authority agent

In [10]:
# Setup for connection with Authority agent
alias = None
auto_accept = False
public = "false" # Do not use public DID
multi_use = "false" # Invitation is only for one invitee

# Await response
invitation_response = await agent_controller.connections.create_invitation(alias, auto_accept, public, multi_use)

# Keep connection_id to use it in other protocols (e.g., messaging)
connection_id = invitation_response["connection_id"]

----------------------------------------------------------
Connection Webhook Event Received
Connection ID :  7a04ff63-a269-44a5-94af-decb75ba51f7
State :  invitation
Routing State :  none
Their Role :  invitee
----------------------------------------------------------
invitation


### 2.2 – Send invitation with Authority
Sharing the invitation can be done via e.g., phone, zoom, E-Mail or QR-code. In this scenario, please copy and paste the invitation into `01_issue_VC_city.ipynb` of the Authority agent (see Step 2.1)

In [11]:
# Get and print invitation
invitation = invitation_response["invitation"]
print("Copy the following dict and past it into the Authority agency")
pprint(invitation)

Copy the following dict and past it into the Authority agency
{
    '@id': '1a4ef9a6-2dc4-4eae-b6b2-6003b1903aa2',
    '@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/connections/1.0/invitation',
    'label': 'City',
    'recipientKeys': ['2HBfxB6MbCXv8tDAriinYY5BzT8gZUtUYKAQ1VXDNDmF'],
    'serviceEndpoint': 'https://80a67f24c0cb.ngrok.io',
}
----------------------------------------------------------
Connection Webhook Event Received
Connection ID :  7a04ff63-a269-44a5-94af-decb75ba51f7
State :  request
Routing State :  none
Their Role :  invitee
----------------------------------------------------------
request


**BREAK POINT:** Please switch to agent `Authority`, open `01_issue_VC_city.ipynb`, and continue with Step 2

---

### 2.3 – Accept request response by Authority agent

In [12]:
# Accept request response of Authority
accept_request_response = await agent_controller.connections.accept_request(connection_id)

----------------------------------------------------------
Connection Webhook Event Received
Connection ID :  7a04ff63-a269-44a5-94af-decb75ba51f7
State :  response
Routing State :  none
Their Role :  invitee
----------------------------------------------------------
response


### 2.4 – Sent trust ping to establish and activate connection

In [13]:
await agent_controller.messaging.trust_ping(connection_id, "send trust ping")

{'thread_id': '7079dec1-8d30-4ec1-9431-edfc11f8f6e9'}

----------------------------------------------------------
Connection Webhook Event Received
Connection ID :  7a04ff63-a269-44a5-94af-decb75ba51f7
State :  active
Routing State :  none
Their Role :  invitee
----------------------------------------------------------
Connection ID: 7a04ff63-a269-44a5-94af-decb75ba51f7 is now active.


## 3 – Request VC from `Authority` agent
### 3.1 – Message `Authority` to request a VC

In [14]:
basic_message = "Hello Authority agent"
await agent_controller.messaging.send_message(connection_id, basic_message)

{}

/tmp/ipykernel_21/3503404827.py:75: RuntimeWarning: coroutine 'MessagingController.send_message' was never awaited
  agent_controller.messaging.send_message(connection_id, "This is a response from Bob")


Handle message 7a04ff63-a269-44a5-94af-decb75ba51f7
{
    'connection_id': '7a04ff63-a269-44a5-94af-decb75ba51f7',
    'content': 'Hello City. Please provide me your city and country.',
    'message_id': '948fb0ff-e570-4eba-a9e4-ae007749db41',
    'sent_time': '2021-08-09 14:19:22.306175Z',
    'state': 'received',
}


**BREAK POINT:** Go to Step 3 in the `01_issue_VC_city.ipynb` notebook of the `Authority` agent.

---

### 3.1 – Provide `Authority` with the relevant information to issue a VC

In [15]:
basic_message = "I would like the get a VC proving that I am an official city agency in Berlin, DE."
await agent_controller.messaging.send_message(connection_id, basic_message)

{}


---------------------------------------------------

Handle Issue Credential Webhook
Connection ID : 7a04ff63-a269-44a5-94af-decb75ba51f7
Credential exchange ID : cbc6367d-3e65-4705-8c7a-e24c5af0d96a
Agent Protocol Role :  holder
Protocol State :  offer_received

---------------------------------------------------

Handle Credential Webhook Payload
Credential Offer Recieved
The proposal dictionary is likely how you would understand and display a credential offer in your application

 {'@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/issue-credential/1.0/propose-credential', '@id': '6341c9c8-6c12-4bdf-858e-59169269f2e9', 'credential_proposal': {'@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/issue-credential/1.0/credential-preview', 'attributes': [{'name': 'city', 'value': 'Berlin'}, {'name': 'country', 'value': 'DE'}, {'name': 'isCityAgency', 'value': 'TRUE'}]}, 'schema_id': 'MTXSc4YD8wixyM9ekZbmMC:2:certify-city-agency:0.0.1', 'comment': 'Issuing VC that City is an agency', 'cred_def_id':

**BREAK POINT:** Return to step 

---


### 3.2 – Request VC from `Authority`'s Offer

Note: Your agent will automatically respond if ACAPY_AUTO_RESPOND_CREDENTIAL_OFFER=true flag is set in .env file of agent. Default is false.

To respond to an offer you must identify the offer using the credential_exchange_id generated for it. This is available from within the issue-credential holder handler. You could add custom logic in this loop `elif state == "request":` to handle this.

However, we will fetch the credential exchange records and **assume** this agent only has one record. Customise accordingly.

In [16]:
# Setup arguments for response
thread_id=None
state = "offer_received"
role = "prover"

records_response = await agent_controller.issuer.get_records(connection_id=connection_id,state=state)
record = records_response["results"][0]
record_id = record["credential_exchange_id"]

In [17]:
# Send request for VC
await agent_controller.issuer.send_request_for_record(record_id)


---------------------------------------------------

Handle Issue Credential Webhook
Connection ID : 7a04ff63-a269-44a5-94af-decb75ba51f7
Credential exchange ID : cbc6367d-3e65-4705-8c7a-e24c5af0d96a
Agent Protocol Role :  holder
Protocol State :  request_sent

---------------------------------------------------

Handle Credential Webhook Payload

A credential request object contains the commitment to the agents master secret using the nonce from the offer


{'trace': False,
 'auto_issue': False,
 'state': 'request_sent',
 'credential_definition_id': 'MTXSc4YD8wixyM9ekZbmMC:3:CL:246685:default',
 'auto_offer': False,
 'credential_exchange_id': 'cbc6367d-3e65-4705-8c7a-e24c5af0d96a',
 'credential_request_metadata': {'master_secret_blinding_data': {'v_prime': '12015492923889721612939121956573559186323965159083211993433006626736462735856072838719299728646968818765086255347183925408187791646665368160937934116797408585592528768792470532612427975420156745961199960469405746846029581686675188546124205443328786885409453320087332573752081886979802768479557966175861181475703915809741477337115363216501003897738362593393866475033240798805279061208312638584835928546729376425757556819132451735962896121539373024968515426089614167457673043182240273005455130994680230174593433020527148814331762604777518024985674933572722255856490374127744180313915246204660298018799879429849694961123785910154059481007385806',
   'vr_prime': None},
  'nonce': '3910890723165795


---------------------------------------------------

Handle Issue Credential Webhook
Connection ID : 7a04ff63-a269-44a5-94af-decb75ba51f7
Credential exchange ID : cbc6367d-3e65-4705-8c7a-e24c5af0d96a
Agent Protocol Role :  holder
Protocol State :  credential_received

---------------------------------------------------

Handle Credential Webhook Payload
Received Credential


### 3.3 – Store received VC in wallet

This will be done automatically if the ACAPY_AUTO_STORE_CREDENTIAL=true flag is set in the .env file for this agent. Default is false.

Again you could handle this in your holder handler function in the `elif state == "credential_received":` loop.

In [18]:
credential_id = "isCityAgency-VC"
store_cred_response = await agent_controller.issuer.store_credential(record_id, credential_id)


---------------------------------------------------

Handle Issue Credential Webhook
Connection ID : 7a04ff63-a269-44a5-94af-decb75ba51f7
Credential exchange ID : cbc6367d-3e65-4705-8c7a-e24c5af0d96a
Agent Protocol Role :  holder
Protocol State :  credential_acked

---------------------------------------------------

Handle Credential Webhook Payload
Credential Stored

{'referent': 'isCityAgency-VC', 'attrs': {'city': 'Berlin', 'country': 'DE', 'isCityAgency': 'TRUE'}, 'schema_id': 'MTXSc4YD8wixyM9ekZbmMC:2:certify-city-agency:0.0.1', 'cred_def_id': 'MTXSc4YD8wixyM9ekZbmMC:3:CL:246685:default'}

The referent acts as the identifier for retrieving the raw credential from the wallet
Referent isCityAgency-VC


## Terminate Controller

Whenever you have finished with this notebook, be sure to terminate the controller. This is especially important if your business logic runs across multiple notebooks.

In [19]:
await agent_controller.terminate()